In [5]:
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import ipywidgets as widgets

## Neural Network Dynamics:

$$\dot{r}_{1} = \gamma[ -r_{1} + tanh(m_{11}r_{1} + m_{12}r_{2} + u_{1})]$$
$$\dot{r}_{2} = \gamma[ -r_{2} + tanh(m_{21}r_{1} + m_{22}r_{2} + u_{2})]$$

In [6]:
def f(t, z, gamma, m11, m12, m21, m22, u1, u2):
    r1, r2 = z
    z_next = np.zeros(2)
    z_next[0] = gamma*(-r1 + np.tanh(m11*r1 + m12*r2 + u1))
    z_next[1] = gamma*(-r2 + np.tanh(m21*r1 + m22*r2 + u2))
    return z_next

## Nullclines:

$$\dot{r}_{1}: r_{2} = \frac{1}{m_{12}}\left(\frac{1}{2}\ln\left(\frac{1+r_1}{1-r_1}\right) - m_{11}r_{1} - u_{1} \right)$$
$$\dot{r}_{2}: r_{1} = \frac{1}{m_{21}}\left(\frac{1}{2}\ln\left(\frac{1+r_2}{1-r_2}\right) - m_{22}r_{2} - u_{2} \right)$$

In [7]:
def r1_null(r1, m11, m12, u1):
    return 1/m12*(1/2*np.log((1+r1)/(1-r1)) - m11*r1 -u1)

def r2_null(r2, m21, m22, u2):
    return 1/m21*(1/2*np.log((1+r2)/(1-r2)) - m22*r2 -u2)

In [8]:
@widgets.interact(gamma=(0, 5, 0.1), m11=(0, 5, 0.1), m12=(0, 5, 0.1), m21=(0, 5, 0.1), m22=(0, 5, 0.1), 
                 u1=(0, 5, 0.1), u2=(0, 5, 0.1), r10=(-1, 1, 0.1), r20=(-1, 1, 0.1))
def update(gamma=0.1, m11=0.1, m12=0.1, m21=0.1, m22=0.1, u1=0.1, u2=0.1, r10=0, r20=0):
    t_span = [0, 100]
    z0 = [r10, r20]
    params = (gamma, m11, m12, m21, m22, u1, u2)
    sln = solve_ivp(f, t_span, z0, args=params, dense_output=True)
    t = np.linspace(0, 100, 10000)
    z = sln.sol(t)
    
    
    fig, ax = plt.subplots(1, 1, figsize=(11,7))
    r_range = np.linspace(-1, 1, 1000)
    ax.plot(r_range, r1_null(r_range, m11, m12, u1))
    ax.plot(r2_null(r_range, m21, m22, u2), r_range)
    ax.plot(z[0], z[1])
    ax.set_xlim(-2, 2)
    ax.set_ylim(-2, 2)

interactive(children=(FloatSlider(value=0.1, description='gamma', max=5.0), FloatSlider(value=0.1, description…

## Plot Vector Field and Nullclines:

In [9]:
def color_map(x):
    colors = np.log10(x)
    return np.nan_to_num(colors, neginf=0) 
    
def phase_portrait(ax, f, t, xlim, ylim, num_pts, args, norm_arrows=True, stream=True, quiver=True):
    
    x = np.linspace(-xlim, xlim, num_pts)
    y = np.linspace(-ylim, ylim, num_pts)
    X, Y = np.meshgrid(x, y)
    u = np.zeros((num_pts, num_pts))
    v = np.zeros((num_pts, num_pts))
    flow_mag = np.zeros((num_pts, num_pts))

    for i in range(num_pts):
        for j in range(num_pts):
            u[i,j], v[i,j] = f(0, [X[i,j], Y[i,j]], *args)
            flow_mag[i,j] = np.sqrt(u[i,j]**2 + v[i,j]**2)
            if (u[i,j], v[i,j]) != (0,0):
                u[i,j] *= 1/flow_mag[i,j]
                v[i,j] *= 1/flow_mag[i,j]
    
    ax.quiver(X[::1, ::1], Y[::1, ::1], u[::1, ::1], v[::1, ::1], color_map(flow_mag[::1, ::1]), cmap="winter", alpha=0.1)
        
    ax.streamplot(X, Y, u, v,  color=color_map(flow_mag), cmap="winter", density=1)
    ax.grid(True, which='both')  

In [16]:
@widgets.interact(gamma=(0, 5, 0.1), m11=(-5, 5, 0.1), m12=(-5, 5, 0.1), m21=(-5, 5, 0.1), m22=(-5, 5, 0.1), 
                u1=(-5, 5, 0.1), u2=(-5, 5, 0.1), r10=(-1, 1, 0.1), r20=(-1, 1, 0.1))
def update(gamma=0.1, m11=0.1, m12=0.1, m21=0.1, m22=0.1, u1=0.1, u2=0.1, r10=0, r20=0):
    t_span = [0, 500]
    z0 = [r10, r20]
    params = (gamma, m11, m12, m21, m22, u1, u2)
    sln = solve_ivp(f, t_span, z0, args=params, dense_output=True)
    t = np.linspace(0, 500, 10000)
    z = sln.sol(t)
    
    
    fig, ax = plt.subplots(1, 1, figsize=(11,7))
    r_range = np.linspace(-1, 1, 10000)
    phase_portrait(ax, f, t, 2, 2, 15, params)
    ax.plot(r_range, r1_null(r_range, m11, m12, u1), 'y', linewidth=2, label='$\dot{r}_{1}=0$')
    ax.plot(r2_null(r_range, m21, m22, u2), r_range, 'r', linewidth=2, label='$\dot{r}_{2}=0$')
    ax.plot(z[0], z[1], 'k', linewidth=2)
    ax.set_xlim(-2, 2)
    ax.set_ylim(-2, 2)
    ax.legend()

interactive(children=(FloatSlider(value=0.1, description='gamma', max=5.0), FloatSlider(value=0.1, description…

## Find Fixed Points:

In [12]:
from scipy.optimize import fsolve

In [13]:
def nullclines_s(r, gamma, m11, m12, m21, m22, u1, u2):
    r1, r2 = r
    return (r1_null(r1, m11, m12, u1) - r1, r2_null(r2, m21, m22, u2) - r2)

def nullclines_o(r, gamma, m11, m12, m21, m22, u1, u2):
    r1, r2 = r
    r_next = np.zeros(2)
    r_next[0] = -r1 + np.tanh(m11*r1 + m12*r2 + u1)
    r_next[1] = -r2 + np.tanh(m21*r1 + m22*r2 + u2)
    return r_next
    

In [14]:
def find_roots(f, d, xrange, yrange, n, args, tol=1e-8):
    x1, x2 = xrange
    y1, y2 = yrange
    x = np.linspace(x1, x2, n)
    y = np.linspace(y1, y2, n)
    xv, yv = np.meshgrid(x, y)
    
    roots = []
    
    for i in range(n):
        for j in range(n):
            root = fsolve(f, [xv[i][j], yv[i][j]], args, maxfev=9000)
            if np.linalg.norm(f(root, *args)) < tol:
                roots.append(root)
    #roots = np.unique(roots, axis=0)
    return roots

In [64]:
@widgets.interact(gamma=(0, 5, 0.1), m11=(0, 5, 0.1), m12=(0, 5, 0.1), m21=(0, 5, 0.1), m22=(0, 5, 0.1), 
                 u1=(0, 5, 0.1), u2=(0, 5, 0.1), r10=(-1, 1, 0.1), r20=(-1, 1, 0.1))
def update(gamma=0.1, m11=0.1, m12=0.1, m21=0.1, m22=0.1, u1=0.1, u2=0.1, r10=0, r20=0):
    t_span = [0, 100]
    z0 = [r10, r20]
    params = (gamma, m11, m12, m21, m22, u1, u2)
    sln = solve_ivp(f, t_span, z0, args=params, dense_output=True)
    t = np.linspace(0, 100, 10000)
    z = sln.sol(t)
    fixed_points = find_roots(nullclines_o, 2, [-1, 1], [-1, 1], 20, params)
    
    fig, ax = plt.subplots(1, 1, figsize=(11,7))
    r_range = np.linspace(-1, 1, 1000)
    ax.plot(r_range, r1_null(r_range, m11, m12, u1))
    ax.plot(r2_null(r_range, m21, m22, u2), r_range)
    ax.plot(z[0], z[1])
    for x in fixed_points:
        ax.plot(x[0], x[1], 'ko')
    ax.set_xlim(-2, 2)
    ax.set_ylim(-2, 2)

interactive(children=(FloatSlider(value=0.1, description='gamma', max=5.0), FloatSlider(value=0.1, description…

## Phase Portrait with Nullclines and Fixed Points:

In [15]:
@widgets.interact(gamma=(0, 5, 0.1), m11=(-5, 5, 0.1), m12=(-5, 5, 0.1), m21=(-5, 5, 0.1), m22=(-5, 5, 0.1), 
                u1=(-5, 5, 0.1), u2=(-5, 5, 0.1), r10=(-1, 1, 0.1), r20=(-1, 1, 0.1))
def update(gamma=0.1, m11=0.1, m12=0.1, m21=0.1, m22=0.1, u1=0.1, u2=0.1, r10=0, r20=0):
    t_span = [0, 500]
    z0 = [r10, r20]
    params = (gamma, m11, m12, m21, m22, u1, u2)
    sln = solve_ivp(f, t_span, z0, args=params, dense_output=True)
    t = np.linspace(0, 500, 10000)
    z = sln.sol(t)
    fixed_points = find_roots(nullclines_o, 2, [-1, 1], [-1, 1], 20, params)
    
    
    fig, ax = plt.subplots(1, 1, figsize=(11,7))
    r_range = np.linspace(-1, 1, 10000)
    phase_portrait(ax, f, t, 2, 2, 15, params)
    ax.plot(r_range, r1_null(r_range, m11, m12, u1), 'y', linewidth=2, label='$\dot{r}_{1}=0$')
    ax.plot(r2_null(r_range, m21, m22, u2), r_range, 'r', linewidth=2, label='$\dot{r}_{2}=0$')
    ax.plot(z[0], z[1], 'k', linewidth=2)
    for x in fixed_points:
        ax.plot(x[0], x[1], 'ko')
    ax.set_xlim(-2, 2)
    ax.set_ylim(-2, 2)
    ax.legend(loc='upper left')

interactive(children=(FloatSlider(value=0.1, description='gamma', max=5.0), FloatSlider(value=0.1, description…

## Eigenvalues of Jacobian at Fixed Points:

$$\dot{r}_{1} = \gamma[ -r_{1} + tanh(m_{11}r_{1} + m_{12}r_{2} + u_{1})] = f_{1}(r_{1}, r_{2})$$
$$\dot{r}_{2} = \gamma[ -r_{2} + tanh(m_{21}r_{1} + m_{22}r_{2} + u_{2})] = f_{2}(r_{1}, r_{2})$$
$$\space$$
$$DF(r_{1}, r_{2}) = \begin{pmatrix}
\frac{\partial f_{1}}{\partial r_{1}} & \frac{\partial f_{1}}{\partial r_{2}}\\
\frac{\partial f_{2}}{\partial r_{1}} & \frac{\partial f_{2}}{\partial r_{2}}
\end{pmatrix} = \begin{pmatrix}
\gamma\left[-1 + m_{11}sech^{2}(m_{11}r_{1} + m_{12}r_{2} + u_{1}) \right] & 
\gamma m_{12}sech^{2}(m_{11}r_{1} + m_{12}r_{2} + u_{1})\\
\gamma m_{21}sech^{2}(m_{21}r_{1} + m_{22}r_{2} + u_{2}) & 
\gamma\left[-1 + m_{22}sech^{2}(m_{21}r_{1} + m_{22}r_{2} + u_{2}) \right]
\end{pmatrix}$$

In [60]:
def jac(r, gamma, m11, m12, m21, m22, u1, u2):
    r1, r2 = r
    j11 = gamma*(-1 + m11*1/np.cosh(m11*r1 + m12*r2 + u1)**2)
    j12 = gamma*m12*1/np.cosh(m11*r1 + m12*r2 + u1)**2
    j21 = gamma*m21*1/np.cosh(m21*r1 + m22*r2 + u2)**2
    j22 = gamma*(-1 + m22*1/np.cosh(m21*r1 + m22*r2 + u2)**2)
    return np.asarray([[j11, j12], [j21, j22]])

In [61]:
gamma = 0.5
m11 = 1.4
m12 = 0.2
m21 = 0.2
m22 = 1.6
u1 = 0
u2 = 0

np.linalg.eig(jac([0, 0], gamma, m11, m12, m21, m22, u1, u2)) 

(array([0.1381966, 0.3618034]),
 array([[-0.85065081, -0.52573111],
        [ 0.52573111, -0.85065081]]))